In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker, relationship, join
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Table, Column, Integer, ForeignKey
from sqlalchemy.orm import relationship
from aquabyte.optics import convert_to_world_point, depth_from_disp, pixel2world, euclidean_distance
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from statsmodels.regression.quantile_regression import QuantReg

from PIL import Image
from copy import copy
from aquabyte.visualize import _normalize_world_keypoints

from scipy.stats import gaussian_kde
# import mpld3
# from mpl_toolkits.mplot3d import Axes3D
import pickle
from PIL import Image, ImageDraw
from multiprocessing import Pool, Manager

import sys
sys.path.append('/root/alok/repos/cv_research/alok/biomass_estimation/production_data_analysis_v3')
# from template_matching import enhance, find_matches_and_homography, adjust_keypoints


pd.set_option('max_columns', 500)
pd.set_option('max_colwidth', 200)

In [6]:
adf = pd.read_csv('/root/data/alok/biomass_estimation/playground/yolo_matches.csv')

In [7]:
# AWS credentials
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")
s3_access_utils = S3AccessUtils('/root/data')


rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from keypoint_annotations where pen_id=61 and captured_at >= '2019-09-13' and captured_at <= '2019-09-20';
"""
mdf = rds_access_utils.extract_from_database(query)


In [11]:
adf.left_crop_url

0        https://s3-eu-west-1.amazonaws.com/aquabyte-cr...
1        https://s3-eu-west-1.amazonaws.com/aquabyte-cr...
2        https://s3-eu-west-1.amazonaws.com/aquabyte-cr...
3        https://s3-eu-west-1.amazonaws.com/aquabyte-cr...
4        https://s3-eu-west-1.amazonaws.com/aquabyte-cr...
                               ...                        
19797    http://aquabyte-crops.s3.eu-west-1.amazonaws.c...
19798    http://aquabyte-crops.s3.eu-west-1.amazonaws.c...
19799    http://aquabyte-crops.s3.eu-west-1.amazonaws.c...
19800    http://aquabyte-crops.s3.eu-west-1.amazonaws.c...
19801    http://aquabyte-crops.s3.eu-west-1.amazonaws.c...
Name: left_crop_url, Length: 19802, dtype: object

In [17]:
len(set(mdf.left_image_url.tolist()).intersection(set(adf.left_crop_url.tolist())))

2295

In [27]:
adf['ground_truth_keypoints'] = ''
for idx, row in adf.iterrows():
    left_crop_url = row.left_crop_url
    if left_crop_url in mdf.left_image_url:
        mask = mdf.left_image_url == left_crop_url
        keypoints = mdf[mask].keypoints.iloc[0]
        
        adf.loc[mask, 'ground_truth_keypoints'] = json.dumps(mdf[mask].keypoints)
    
adf.loc[adf.left_crop_url.isin(mdf.left_image_url), 'ground_truth_keypoints'] = json.dumps()

(4590, 22)

In [26]:
adf.head(10).left_crop_url

0    https://s3-eu-west-1.amazonaws.com/aquabyte-crops/environment=production/site-id=40/pen-id=61/date=2019-09-13/hour=00/at=2019-09-13T00:04:36.127087000Z/left_frame_crop_1622_1016_4074_1977.jpg
1    https://s3-eu-west-1.amazonaws.com/aquabyte-crops/environment=production/site-id=40/pen-id=61/date=2019-09-13/hour=00/at=2019-09-13T00:04:36.127087000Z/left_frame_crop_1622_1016_4074_1977.jpg
2       https://s3-eu-west-1.amazonaws.com/aquabyte-crops/environment=production/site-id=40/pen-id=61/date=2019-09-13/hour=00/at=2019-09-13T00:07:07.959382000Z/left_frame_crop_70_342_3114_1409.jpg
3       https://s3-eu-west-1.amazonaws.com/aquabyte-crops/environment=production/site-id=40/pen-id=61/date=2019-09-13/hour=00/at=2019-09-13T00:07:07.959382000Z/left_frame_crop_70_342_3114_1409.jpg
4      https://s3-eu-west-1.amazonaws.com/aquabyte-crops/environment=production/site-id=40/pen-id=61/date=2019-09-13/hour=00/at=2019-09-13T00:07:09.032117000Z/left_frame_crop_974_571_3762_1602.jpg
5      https://